In [1]:
MAX_TIME = 720
INIT_SIZE = 20
CLASSIFICATION = True
REGRESSION = False
MOONS = False
MAKE_CLF = False

In [2]:
import pickle
import socket
from datetime import datetime
from dateutil.tz import tzlocal
start_time = datetime.now(tzlocal())
HOSTNAME = socket.gethostname().split(".")[0]
experiment_name = '11-torch' + "_" + HOSTNAME + "_" + str(MAX_TIME) + "min_" + str(INIT_SIZE) + "init_" + str(start_time).split(".", 1)[0].replace(' ', '_')
experiment_name = experiment_name.replace(':', '-')
experiment_name

'11-torch_s7cfd16_720min_20init_2023-04-27_15-16-12'

# Chapter 11: Sequential Parameter Optimization
## Hyperparameter Tuning: pytorch wth cifar10 Data 

* This notebook exemplifies hyperparameter tuning with SPOT (spotPython).
* The hyperparameter software SPOT was developed in R (statistical programming language), see Open Access book "Hyperparameter Tuning for Machine and Deep Learning with R - A Practical Guide", available here: [https://link.springer.com/book/10.1007/978-981-19-5170-1](https://link.springer.com/book/10.1007/978-981-19-5170-1).


In [3]:
pip list | grep  "spot[RiverPython]"

spotPython                   0.0.44
spotRiver                    0.0.79
Note: you may need to restart the kernel to use updated packages.


In [4]:
# import sys
# !{sys.executable} -m pip install --upgrade build
# !{sys.executable} -m pip install --upgrade --force-reinstall spotPython


## Example 1: HATR Hyperparameter

In [5]:
import torch
from tabulate import tabulate
import copy
import warnings
import numbers
import json
import calendar
import math
import datetime as dt
import numpy as np
from math import inf
import pandas as pd

from scipy.optimize import differential_evolution

import matplotlib.pyplot as plt

from spotPython.spot import spot
from spotPython.hyperparameters.values import (
    add_core_model_to_fun_control,
    assign_values,
    convert_keys,
    get_bound_values,
    get_default_hyperparameters_for_core_model,
    get_default_hyperparameters_for_fun,
    get_default_values,
    get_dict_with_levels_and_types,
    get_values_from_dict,
    get_var_name,
    get_var_type,
    iterate_dict_values,
    modify_hyper_parameter_levels,
    modify_hyper_parameter_bounds,
    replace_levels_with_positions,
    return_conf_list_from_var_dict,
    get_one_sklearn_model_from_X)
from spotPython.hyperparameters.prepare import (
    transform_hyper_parameter_values,
    get_dict_with_levels_and_types,
    convert_keys,
    iterate_dict_values,
)

from spotPython.utils.convert import class_for_name
from spotPython.utils.eda import (
    get_stars,
    gen_design_table)
from spotPython.utils.transform import transform_hyper_parameter_values

from spotPython.data.torch_hyper_dict import TorchHyperDict
from spotPython.fun.hypertorch import HyperTorch
from spotPython.utils.convert import get_Xy_from_df
from spotPython.plot.validation import plot_cv_predictions, plot_roc, plot_confusion_matrix
from spotPython.torch.net import Net_CIFAR10

warnings.filterwarnings("ignore")

In [6]:
print(torch.__version__)
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    print("MPS device: ", mps_device)

2.0.0+cu117
MPS not available because the current PyTorch install was not built with MPS enabled.


## 0. Initialization of the Empty `fun_control` Dictionary

In [7]:
fun_control = {}

## 1. Load Data: Random Data

## Regression

In [8]:
if REGRESSION:
    n_samples = 250
    target_column = "y"
    n_train = 0.6 * n_samples
    n_features = 50
    # Create a random dataset
    X, y = make_regression(n_samples=n_samples, n_features=n_features, noise=1, random_state=42)
    # take X and y and make a pandas dataframe with column names X1, X2, y
    df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))))
    df.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
    # split into train and test
    train = df.iloc[:int(n_train), :]
    test = df.iloc[int(n_train):, :]

## Classification

In [9]:
if MOONS:
    n_features = 2
    n_samples = 250
    ds =  make_moons(n_samples, noise=0.5, random_state=0)
    X, y = ds
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.4, random_state=42
    )
    train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1))))
    test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1))))
    train.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
    test.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
    train.head()

In [10]:
if MAKE_CLF:
    X, y = make_classification(
        n_samples=n_samples,
        n_features=n_features,
        n_informative=40,
        n_redundant=2,
        n_repeated=1,
        n_classes=2,
        flip_y=0.25,
        random_state=0,
        class_sep=0.025,
        shuffle=False,
    )
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

    train = pd.DataFrame(np.hstack((X_train, y_train.reshape(-1, 1))))
    test = pd.DataFrame(np.hstack((X_test, y_test.reshape(-1, 1))))
    train.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
    test.columns = [f"x{i}" for i in range(1, n_features+1)] + [target_column]
    train.head()


## Load CIFAR Data 

In [11]:
from functools import partial
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms

In [12]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [13]:
train, test = load_data()

Files already downloaded and verified
Files already downloaded and verified


In [14]:
train.data.shape, test.data.shape
train

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [15]:
n_samples = len(train)
# add the dataset to the fun_control
fun_control.update({"data": None, # dataset,
               "train": train,
               "test": test,
               "n_samples": n_samples,
               "target_column": None})


## 2. Specification of the Preprocessing Model

In [16]:
# categorical_columns = []
# one_hot_encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
# prep_model = ColumnTransformer(
#         transformers=[
#             ("categorical", one_hot_encoder, categorical_columns),
#         ],
#         remainder=StandardScaler(),
#     )
prep_model = None
fun_control.update({"prep_model": prep_model})

## 3. Select `algorithm` and `core_model_hyper_dict`

In [17]:
# core_model  = RidgeCV
core_model = Net_CIFAR10
fun_control = add_core_model_to_fun_control(core_model=core_model,
                              fun_control=fun_control,
                              hyper_dict=TorchHyperDict,
                              filename=None)

## 4. Modify `hyper_dict` Hyperparameters for the Selected Algorithm aka `core_model`

### Modify hyperparameter of type factor

In [18]:
# fun_control = modify_hyper_parameter_levels(fun_control, "leaf_model", ["LinearRegression"])
# fun_control["core_model_hyper_dict"]

### Modify hyperparameter of type numeric and integer (boolean)

In [19]:
# fun_control = modify_hyper_parameter_bounds(fun_control, "delta", bounds=[1e-10, 1e-6])
# fun_control = modify_hyper_parameter_bounds(fun_control, "min_samples_split", bounds=[3, 20])
#fun_control = modify_hyper_parameter_bounds(fun_control, "merit_preprune", bounds=[0, 0])
# fun_control["core_model_hyper_dict"]

## 5. Selection of the Objective (Loss) Function

There are two metrics:

    1. `metric` is used for the river based evaluation via `eval_oml_iter_progressive`.
    2. `metric_sklearn` is used for the sklearn based evaluation.

In [20]:
fun = HyperTorch(seed=123, log_level=50).fun_torch
weights = 1.0
horizon = 7*24
oml_grace_period = 2
step = 100
weight_coeff = 1.0

fun_control.update({
               "data_dir": None,
               "checkpoint_dir": None,
               "horizon": horizon,
               "oml_grace_period": oml_grace_period,
               "weights": weights,
               "step": step,
               "log_level": 50,
               "weight_coeff": weight_coeff,
               "metric": None,
               "metric_sklearn": None
               })

## 6. Calling the SPOT Function

### Prepare the SPOT Parameters

* Get types and variable names as well as lower and upper bounds for the hyperparameters.

In [21]:
var_type = get_var_type(fun_control)
var_name = get_var_name(fun_control)
fun_control.update({"var_type": var_type,
                    "var_name": var_name})

lower = get_bound_values(fun_control, "lower")
upper = get_bound_values(fun_control, "upper")

In [22]:
print(gen_design_table(fun_control))

| name       | type   |   default |   lower |   upper |
|------------|--------|-----------|---------|---------|
| l1         | int    |     5     |  2      |     9   |
| l2         | int    |     5     |  2      |     9   |
| lr         | float  |     0.001 |  0.0001 |     0.1 |
| batch_size | int    |     4     |  1      |     4   |
| epochs     | int    |     3     |  1      |     4   |


### Run the `Spot` Optimizer

* Run SPOT for approx. x mins (`max_time`).
* Note: the run takes longer, because the evaluation time of initial design (here: `initi_size`, 20 points) is not considered.

In [23]:
spot_torch = spot.Spot(fun=fun,
                   lower = lower,
                   upper = upper,
                   fun_evals = inf,
                   fun_repeats = 1,
                   max_time = MAX_TIME,
                   noise = False,
                   tolerance_x = np.sqrt(np.spacing(1)),
                   var_type = var_type,
                   var_name = var_name,
                   infill_criterion = "y",
                   n_points = 1,
                   seed=123,
                   log_level = 50,
                   show_models= False,
                   show_progress= True,
                   fun_control = fun_control,
                   design_control={"init_size": INIT_SIZE,
                                   "repeats": 1},
                   surrogate_control={"noise": True,
                                      "cod_type": "norm",
                                      "min_theta": -4,
                                      "max_theta": 3,
                                      "n_theta": len(var_name),
                                      "model_optimizer": differential_evolution,
                                      "model_fun_evals": 10_000,
                                      "log_level": 50
                                      })
spot_torch.run()

We will use 4 GPUs!
Using cuda device
[1,  2000] loss: 2.345
[1,  4000] loss: 1.174
[1,  6000] loss: 0.783
[1,  8000] loss: 0.587
[1, 10000] loss: 0.470
[2,  2000] loss: 2.348
[2,  4000] loss: 1.173
[2,  6000] loss: 0.783
[2,  8000] loss: 0.587
[2, 10000] loss: 0.470
[3,  2000] loss: 2.348
[3,  4000] loss: 1.176
[3,  6000] loss: 0.781
[3,  8000] loss: 0.587
[3, 10000] loss: 0.469
[4,  2000] loss: 2.345
[4,  4000] loss: 1.174
[4,  6000] loss: 0.783
[4,  8000] loss: 0.586
[4, 10000] loss: 0.470
[5,  2000] loss: 2.347
[5,  4000] loss: 1.173
[5,  6000] loss: 0.782
[5,  8000] loss: 0.586
[5, 10000] loss: 0.469
[6,  2000] loss: 2.346
[6,  4000] loss: 1.176
[6,  6000] loss: 0.783
[6,  8000] loss: 0.587
[6, 10000] loss: 0.471
[7,  2000] loss: 2.349
[7,  4000] loss: 1.173
[7,  6000] loss: 0.782
[7,  8000] loss: 0.587
[7, 10000] loss: 0.469
[8,  2000] loss: 2.346
[8,  4000] loss: 1.175
[8,  6000] loss: 0.783
[8,  8000] loss: 0.586
[8, 10000] loss: 0.470
Accuracy of the network on the validation 

### 4 Results

In [ ]:
SAVE = True
LOAD = False

if SAVE:
    result_file_name = "res_" + experiment_name + ".pkl"
    with open(result_file_name, 'wb') as f:
        pickle.dump(spot_torch, f)

if LOAD:
    result_file_name = "res_ch10-friedman-hpt-0_maans03_60min_20init_1K_2023-04-14_10-11-19.pkl"
    with open(result_file_name, 'rb') as f:
        spot_torch =  pickle.load(f)

* Show the Progress of the hyperparameter tuning:

In [ ]:
spot_torch.plot_progress(log_y=False, filename="../Figures.d/" + experiment_name+"_progress.pdf")



* Print the Results

In [ ]:
res = spot_torch.print_results(print_screen=False)
print(tabulate(
   res,
   headers=["Parameter", "Value"],
   numalign="right",
   tablefmt="github")
)

## Show variable importance

In [ ]:
spot_torch.plot_importance(threshold=0.025, filename="../Figures.d/" + experiment_name+"_importance.pdf")

In [ ]:
print(gen_design_table(fun_control=fun_control, spot=spot_torch))

## Get Default Hyperparameters

In [ ]:
values_default = get_default_hyperparameters_for_core_model(fun_control=fun_control,
                                                   hyper_dict=TorchHyperDict)
values_default

In [ ]:
model_default = fun_control["core_model"](**values_default)


## Get SPOT Results

In [ ]:
X = spot_torch.to_all_dim(spot_torch.min_X.reshape(1,-1))
model_spot = get_one_sklearn_model_from_X(X, fun_control)

## Test Set Accuracy

In [ ]:
model_default.test_accuracy(fun_control=fun_control)

In [ ]:
model_spot.test_accuracy(fun_control=fun_control)

In [ ]:
min(spot_torch.y), max(spot_torch.y)

## Detailed Hyperparameter Plots

* For productive use, you might want to select:
  * `min_z=min(spot_torch.y)` and
  * `max_z = max(spot_torch.y)`
* These settings are not so colorful as visualizations that use `None` for the ranges, but give better insights.

In [ ]:
threshold = 0.025
impo = spot_torch.print_importance(threshold=threshold, print_screen=True)
var_plots = [i for i, x in enumerate(impo) if x[1] > threshold]
min_z = min(spot_torch.y)
max_z = max(spot_torch.y)
n = spot_torch.k
for i in var_plots:
    for j in var_plots:
        if j > i:
            filename = "../Figures.d/" + experiment_name+"_contour_"+str(i)+"_"+str(j)+".pdf"
            spot_torch.plot_contour(i=i, j=j, min_z=min_z, max_z = max_z, filename=filename)

## Plot all Combinations of Hyperparameters

* Warning: this may take a while.

In [ ]:
PLOT_ALL = False
if PLOT_ALL:
    n = spot_torch.k
    for i in range(n-1):
        for j in range(i+1, n):
            spot_torch.plot_contour(i=i, j=j, min_z=min_z, max_z = max_z)